In [4]:
__author__ = "Hieu T Nguyen"
# Water Flow via Geometric Programming
#https://agupubs.onlinelibrary.wiley.com/doi/epdf/10.1002/2014WR016756
        
import cvxpy as cp
import numpy as np
import math 

#Example of Disciplined Geometric programming in cvxpy
#https://www.cvxpy.org/examples/index.html#dgp-examples

#=================================================================================
#  WE ARE NOW READY FOR WATER POWER FLOW VIA GP 

def dict_key(dict, index):
    return list(dict)[index]


c =[10, 1, 1]    
m =[3,  1, 1 ]

I = 8; #0 is source (number of nodes)
from_node = [0, 1 , 2, 3, 3, 7,  2 ]
to_node =   [1, 2 , 3, 4, 5, 6,  7 ]
end_node =  [4, 5, 6]

demand = dict((key, 1000) for key in end_node) 

arc = dict((key, [from_node[key], to_node[key]]) for key in range(len(from_node)))       

pipe =     {1: [1, 2],
            2: [2, 3],
            3: [3, 4],
            4: [3, 5],
            5: [7, 6]}
pump =     {1: [0,1]}


elevation = {1: 100, 2: 100, 3: 100, 4: 105, 5: 100, 6: 0, 7: 50}

valve ={1: [2,7]}; V= len(valve)

P = len(pipe) # number of pipe
diameter = {1: 16, 2: 12, 3: 8, 4: 8, 5: 8}#ft
pipe_length =1000 #ft
rough_coff =100
#https://en.wikipedia.org/wiki/Hazen–Williams_equation US imperial units
R = dict((k, 4.52*2.307/(rough_coff**1.852)/(diameter[k]**4.8704)*pipe_length) 
         for k in range(1,len(pipe)+1)) #Hazen–Williams coff
#Head node limits
P_min = 40; P_max = 200
H_max = dict((key, elevation[key]+P_max) for key in range(1,I)) 
H_min = dict((key, elevation[key]+P_min) for key in range(1,I))

print(H_min)

H = cp.Variable(I+1, pos=True)
h = cp.Variable(P+1, pos=True)
gamma = cp.Variable(V+1, pos=True)
beta  = cp.Variable(len(pump)+1, pos=True)

q= cp.Variable(len(from_node), pos=True)
inflow  = [0 for i in range(I)]
outflow = [0 for i in range(I)]
inflow[0] = cp.Variable(1, pos=True)
for k in range(len(arc)):
    [i,j] = arc[k]
    outflow[i] +=q[k]
    inflow[j] = q[k]
for i in end_node:
    outflow[i] = cp.Variable(1, pos=True)
initial = [H[0]==100] #source node's head is 100 ft   
initial += [outflow[i] >=1000 for i in end_node] 
head_limit  =[ H[i] <= H_max[i] for i in range(1,I)]
head_limit +=[ H[i] >= H_min[i] for i in range(1,I)]  

pipe_head_loss  =[H[pipe[k][1]] + h[k] <=  H[pipe[k][0]] for k in pipe] 

valve_head =[H[valve[k][0]]*gamma[k] == H[valve[k][1]] for k in valve]

valve_lim = [gamma[k] <= 0.5 for k in valve]

pump_head = [H[pump[k][0]]*beta[k] == H[pump[k][1]] for k in pump] 
pump_lim   =[beta[k] <= 100 for k in pump ] 
pump_lim   +=[ beta[k] >= 1  for k in pump ] 

flow_balance =[inflow[i] >= outflow[i] for i in range(I)]
head_loss = [h[k]== R[k]*(q[k]**1.852) for k in pipe ]

#objective 
for i in range(2,7):
    if i==2:
        obj3=c[2]*(H[i]**(-m[2]))
    else:
        obj3+=c[2]*(H[i]**(-m[2]))
        
t =cp.Variable(pos=True)
objective = [c[0]*(beta[1]**m[0]) + c[1]*(gamma[1]**(m[1]))+ obj3 <= t]


constraint = (initial + head_limit + pipe_head_loss+ valve_head 
         +valve_lim+pump_head+pump_lim+flow_balance + 
          head_loss + objective)

problem = cp.Problem(cp.Minimize(t), constraint)
problem.solve(gp=True, solver ='MOSEK',  verbose=True, warm_start = True)

print('=================================================================')
print('obj:', t.value)
print()
print('Head node:')
for i in range(1,I): print('node', i, ':', H.value[i]) 

for k in valve: print('gamma of valve', k, ':', gamma.value[k]) 

for k in pump: print('beta of pump', k, ':', beta.value[k]) 

print('=================================================================')

for k in pipe: print('pipe flow', k, ':', q.value[k]) 

print('=================================================================')

print("check if relaxation is binding (dual value >0)")
for i in range(len(pipe)): print(pipe_head_loss[i].dual_value) 

{1: 140, 2: 140, 3: 140, 4: 145, 5: 140, 6: 40, 7: 90}


Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 60              
  Cones                  : 21              
  Scalar variables       : 113             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 60              
  Cones                  : 21              
  Scalar variables       : 113             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 32
Optimizer  - Co